In [1]:
import numpy as np
import pandas as pd
from scipy.stats import beta
from scipy.optimize import minimize

In [4]:
def betaEst(y,w,weights):
    y_idx = np.where(~np.isnan(y))
    y = y[y_idx]
    w = w[y_idx]
    weights = weights[y_idx]
    N = sum(weights*w)
    p = sum(weights*w*y)/N
    v = sum(weights*w*y*y)/N - p*p
    logab = np.log(np.array([p,1-p])) + np.log(np.maximum(1E-6,(p*(1-p)/v-1)))
    if ( sum(~np.isnan(y))==2):
        return np.exp(logab)

    opt = minimize(betaObjf, logab, args=(y,w,weights),method="BFGS")
    return np.exp(opt.x)
             

In [11]:
def betaObjf(logab,ydata,wdata,weights):
    ab = np.exp(logab)
    l = -sum(wdata*weights*beta.logpdf(ydata,ab[0],ab[1]))
    return l


In [12]:
def blc (Y, w, maxiter=25, tol=1E-6, weights=None, verbose=True):
    Ymn = np.min(Y[np.where(Y>0)])
    Ymx = np.max(Y[np.where(Y<1)])
    Y = np.maximum(Y,Ymn/2)
    Y = np.minimum(Y,1-(1-Ymx)/2)
       
    Yobs = np.where(~np.isnan(testY),True,False)

    Ydf = pd.DataFrame(Y)
    Wdf = pd.DataFrame(w)

    
    J = Ydf.shape[1]
    K = Wdf.shape[1]
    n = Wdf.shape[0]   

    
    if ( n != Ydf.shape[0]):
        return print("Dimensions of w and Y do not agree")
    
    if ( weights == None or weights == 0 ):
        weights = np.repeat(1,n)
    mu = np.full([K,J],np.Inf)
    b = np.full([K,J],np.Inf)
    a = np.full([K,J],np.Inf)

    crit = np.Inf
    for i in range(maxiter):
        weightedDF = (weights*2)
        eta = ( np.apply_over_axes(sum,w*weights,0) ) / sum(weights)
        mu0 = np.copy(mu) 
        for k in range(K):
            for j in range(J):
                ab = betaEst(Y[:,j],w[:,k], weights)
                a[k,j] = ab[0]
                b[k,j] = ab[1]
                mu[k,j] = ab[0]/sum(ab)
        ww = np.zeros((n,J,K))
        for k in range(K):
            for j in range(J):
                ww[k,Yobs[:,j],j] = beta.logpdf(Y[Yobs[:,j],j],a[k,j], b[k,j])
        w = np.apply_along_axis(np.sum,2,ww)
        w = w.transpose()
        wmax = np.apply_along_axis(np.max,1,w)
        for k in range(K):
            w[:,k] = w[:,k] - wmax
        w =  (np.transpose(np.exp(w)) * eta)
        w = np.transpose(w)
        like = np.apply_along_axis(sum,1,w)
        w = (1/like) * w
        llike = weights * (np.log(like) + wmax)
        verbose = 1
        crit = np.max(abs(mu-mu0))
        if( verbose == 1):
            if ( crit<tol ):
                break
    return [("a",a),("b",b),("eta",eta),("mu",mu),("w",w),("sum llike",sum(llike))]